In [1]:
import sqlite3
from tqdm import tqdm_notebook as tqdm
from os import *
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from gurobipy import *
from itertools import *
# Set to choose year from which to draw data
year = 2017


In [2]:
# Query database for morning subway data in the given year
conn = sqlite3.connect("resources/data.db")
cur = conn.cursor()
cur.execute(f'''
    SELECT name, entries FROM Subways
    WHERE 7 < CAST(STRFTIME("%H", time) AS INT) 
    AND CAST(STRFTIME("%H", time) AS INT) < 11
    AND CAST(STRFTIME("%Y", time) AS INT) = {year}
''')

# Load into dictionary by station name
data = cur.fetchall()
stations_am = {}
for name, count in tqdm(list(data)):
    if name not in stations_am:
        stations_am[name] = 0
    stations_am[name] += count


HBox(children=(IntProgress(value=0, max=1402617), HTML(value='')))

In [3]:
# Query the afternoon data
conn = sqlite3.connect("resources/data.db")
cur = conn.cursor()
cur.execute(f''' 
    SELECT name, entries FROM Subways
    WHERE 15 < CAST(STRFTIME("%H", time) AS INT) 
    AND CAST(STRFTIME("%H", time) AS INT) < 19
    AND CAST(STRFTIME("%Y", time) AS INT) = {year}
''')

# Load into dictionary by station name
data = cur.fetchall()
stations_pm = {}
for name, count in tqdm(list(data)):
    if name not in stations_pm:
        stations_pm[name] = 0
    stations_pm[name] += count


HBox(children=(IntProgress(value=0, max=1351004), HTML(value='')))

In [4]:
# Simple function to match different labels for same subway station
def fuzzy_match(a, b):
    i = 0
    j = 0
    count = 0
    match = -1
    while True:
        if i >= len(a):
            break
        elif j >= len(b):
            i += 1
            j = match + 1
        elif a[i] == b[j]:
            match = j 
            i, j, count = [x + 1 for x in (i, j, count)]
        else:
            j += 1
    return count / min(len(a), len(b))


In [5]:
import io
import re

# Scaling for longitude to latitude
long_scale = 0.758

# Get station locations on map 
cur.execute(f'''
    SELECT name, longitude * {long_scale}, latitude FROM Stations
''')


# Find the most similar string in given a list to search
def convert_name(string, searchlist):
    string = string.upper()
    m1 = re.search('([0-9]+)', string)
    get_group = lambda m: m if not m else m.group(1)
    candidates = [name.upper() for name in searchlist 
                  if (not m1 or m1.group(1) == get_group(re.search('([0-9]+)', name)))
                     and (m1 or not re.search('([0-9]+)', name))]
    found = max(candidates, key=lambda x: fuzzy_match(string, x)) if candidates else string
    return found

# Stations to coordinates
location_names = {a.upper(): (b, c) for a, b, c in list(cur.fetchall())}
# Create dictionary mapping turnstile-data station names to locations
name_to_location = {}
for k in tqdm(stations_am):
    location_label = convert_name(k, location_names)
    if location_label in location_names:
        name_to_location[k] = location_names[location_label]
        
print(f'{sum(a in name_to_location for a in stations_am) / len(stations_am)*100:.1f}%')


HBox(children=(IntProgress(value=0, max=379), HTML(value='')))


100.0%


In [6]:
from math import sqrt
from scipy.spatial import KDTree

# Number of taxi trips to sample
taxi_rows = 2000000

cur.execute(f'''
    SELECT pickup_longitude * {long_scale}, pickup_latitude,
           dropoff_longitude * {long_scale}, dropoff_latitude 
    FROM Taxis
    WHERE 7 < CAST(STRFTIME("%H", pickup_datetime) AS INT) < 11
    LIMIT {taxi_rows}
''')

def dist(a, b):
    return sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

# Get pickup and dropoff locations
taxis = list(cur.fetchall())

# Zero out matrix representing point to point taxi traffic near subway stations
traffic_matrix = np.zeros([len(name_to_location), len(name_to_location)])

# Iterate through start (a, b) and end (c, d) points
for a, b, c, d in tqdm(taxis):
    # Divide start and end locations into probability distributions such that the value for each station
    # is 1 / (sum(1 / dist(loc, s) for s in stations) / dist(loc, station))
    start = np.array([1 / dist([a, b], name_to_location[x]) for x in name_to_location])
    start = start / start.sum()
    end = np.array([1 / dist([c, d], name_to_location[x]) for x in name_to_location])
    end = end / end.sum()
    # Update point to point matrix with outer product of start and end distributions
    update = start[:, np.newaxis] @ end[np.newaxis, :]
    traffic_matrix += update

# Create dictionary containing matrix values
taxi_traffic = {k: {v: traffic_matrix[i][j] for j, v in enumerate(name_to_location)} for i, k in enumerate(name_to_location)}

HBox(children=(IntProgress(value=0, max=2000000), HTML(value='')))

In [7]:
'''
Creates a matrix p_matrix such that p_matrix[i][j] is the probability of a passenger
leaving the ith station and going to the jth one. The constraints are that the number
of entries to each station is equal to the morning entries and the number of exits is equal
to afternoon entries (presumed to be equal). The diagonal (self traffic) is also set to
zero. The minimized objective function is the sum of the squares between the p_matrix and
the normalized point to point taxi matrix.
'''
m = Model()
station_tuples = [(i, stations_am[i], stations_pm[i], name_to_location[i]) 
                  for i in stations_am if i in name_to_location and i in stations_pm]
station_to_traffic = {a: (b, c) for a, b, c, d in station_tuples}

# Normalize am and pm traffic so sum is 1
station_traffic_am = np.array([v1 for i, (v1, v2) in station_to_traffic.items()])
station_traffic_am = station_traffic_am / station_traffic_am.sum()
station_traffic_pm = np.array([v2 for i, (v1, v2) in station_to_traffic.items()])
station_traffic_pm = station_traffic_pm / station_traffic_pm.sum()

# Non-normalized station flow
x_matrix = np.array([[None for i in station_to_traffic] for j in station_to_traffic])

# Normalized taxi traffic (sum of rows is 1)
target_matrix = np.array([[taxi_traffic[j][i] for i in station_to_traffic] 
                          for j in station_to_traffic]).astype(np.float64)

for i, v in enumerate(target_matrix):
    if v.sum() == 0:
        target_matrix[i] = np.ones(v.shape)

for i, row in enumerate(target_matrix):
    target_matrix[i] = row / row.sum()

for (i, a), (j, b) in product(enumerate(station_to_traffic), enumerate(station_to_traffic)):
    x_matrix[i][j] = m.addVar(vtype=GRB.CONTINUOUS, name=f'{a}->{b}')

# Normalized x_matrix so that the sum of rows is 1
p_matrix = np.array([x_matrix[i] / station_traffic_pm[i] for i in range(len(x_matrix))])

# Sum of sqared difference between p_matrix and target_matrix
objective = ((p_matrix - target_matrix) * (p_matrix - target_matrix)).sum()
m.setObjective(objective, GRB.MINIMIZE)

# Contraints on rows, columns, and diagonal
m.addConstrs(x_matrix[i, :].sum() == station_traffic_pm[i] for i in range(len(x_matrix)))
m.addConstrs(x_matrix[:, j].sum() == station_traffic_am[j] for j in range(len(x_matrix)))
m.addConstrs(x_matrix[k][k] == 0 for k in range(len(x_matrix)))

m.optimize()
solution = np.vectorize(lambda x: x.getValue())(p_matrix)

print(solution)
print('objective = ', objective.getValue())

Academic license - for non-commercial use only
Optimize a model with 1134 rows, 142884 columns and 286146 nonzeros
Model has 142884 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 9e+02]
  QObjective range [3e+03, 4e+09]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-09, 3e-02]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 379 rows and 378 columns
Presolve time: 0.14s
Presolved: 755 rows, 142506 columns, 284635 nonzeros
Presolved model has 142506 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.421e+05
 Factor NZ  : 2.854e+05 (roughly 60 MBytes of memory)
 Factor Ops : 1.437e+08 (less than 1 second per iteration)
 Threads    : 2

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.73705994e+10 -3.73709779e+10  3.77e+05 0

In [8]:
'''
Same as above, but rather than using a row-normalized p_matrix, the non normalized flows are
used (so the sum of rows is not 1). 
'''
m = Model()
station_tuples = [(i, stations_am[i], stations_pm[i], name_to_location[i]) 
                  for i in stations_am if i in name_to_location and i in stations_pm]

station_to_traffic = {a: (b, c) for a, b, c, d in station_tuples}
station_traffic_am = np.array([v1 for i, (v1, v2) in station_to_traffic.items()])
# PM traffic is normalized to have same sum as AM entries
station_traffic_pm = np.array([v2 for i, (v1, v2) in station_to_traffic.items()])
station_traffic_pm = station_traffic_pm / station_traffic_pm.sum() * station_traffic_am.sum()
x_matrix = np.array([[None for i in station_to_traffic] for j in station_to_traffic])
target_matrix_nn = np.array([[taxi_traffic[j][i] for i in station_to_traffic] 
                          for j in station_to_traffic]).astype(np.float64)

# Target matrix is normalized to have same sum as AM entries
target_matrix_nn = target_matrix_nn / target_matrix_nn.sum() * station_traffic_am.sum()

for (i, a), (j, b) in product(enumerate(station_to_traffic), enumerate(station_to_traffic)):
    x_matrix[i][j] = m.addVar(vtype=GRB.CONTINUOUS, name=f'{a}->{b}')

# Minimize sum of squares
objective = ((x_matrix - target_matrix_nn) * (x_matrix - target_matrix_nn)).sum()
m.setObjective(objective, GRB.MINIMIZE)

# Add constraints on rows, columns, and diagonal
m.addConstrs(x_matrix[i, :].sum() == station_traffic_pm[i] for i in range(len(x_matrix)))
m.addConstrs(x_matrix[:, j].sum() == station_traffic_am[j] for j in range(len(x_matrix)))
m.addConstrs(x_matrix[k][k] == 0 for k in range(len(x_matrix)))

m.optimize()
solution_x = np.vectorize(lambda x: x.x)(x_matrix)

print(solution_x)
print('objective = ', objective.getValue())

Optimize a model with 1134 rows, 142884 columns and 286146 nonzeros
Model has 142884 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 6e+04]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 8e+06]
Presolve removed 379 rows and 378 columns
Presolve time: 0.12s
Presolved: 755 rows, 142506 columns, 284635 nonzeros
Presolved model has 142506 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.421e+05
 Factor NZ  : 2.854e+05 (roughly 60 MBytes of memory)
 Factor Ops : 1.437e+08 (less than 1 second per iteration)
 Threads    : 2

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.36988010e+12 -3.53246255e+12  3.39e+06 1.42e+04  4.31e+08     0s
   1   1.80068982e+12 -3.25473048e+12  7.14e+05 7.28e+03  1.18e+08     0s
   2   1.92716426e+12 -9.72964976e+11  1.01e+05 8.81e+02

In [9]:
from math import sqrt
import itertools
from matplotlib.patches import Rectangle
import matplotlib as mpl

# Makes scatterplots of the relationship between each entry in the target_matrix and the solution matrix
# If working correctly, this will show a positive correlation
def make_scatter(xmat, ymat, file):
    %matplotlib inline
    plt.close('all')
    mpl.rcParams.update(mpl.rcParamsDefault)
    eps = 1e-4
    scatpts = [[a, b] for a, b in zip(chain(*xmat), chain(*ymat))]
    norm = [[a, b] for a, b in scatpts if b > eps and a > eps]
    a, b = np.polyfit(*zip(*norm), 1)
    r = np.corrcoef(*zip(*norm))[1, 0]
    extra = Rectangle((0, 0), 1, 1, fc="w", fill=False, edgecolor='none', linewidth=0)
    fit = [a + b * i for i in list(zip(*scatpts))[0]]
    plt.axis('on')
    plt.legend([extra], [f"r = {r}"])
    plt.scatter(*zip(*scatpts), alpha=0.1, s=0.3)
    plt.xlabel('Taxi Flow')
    plt.ylabel('Projected Subway Flow')
    plt.plot(list(zip(*scatpts))[0], fit, color="r", alpha=0.7)
    plt.savefig(f'images/subway_model/{file}_{year}.png')
    
# Make scatterplots for the probability matrix and non-normalized flow
make_scatter(target_matrix, solution, 'subway_scatter')
make_scatter(target_matrix_nn, solution_x, 'subway_nn_scatter')

In [10]:
import json

# Get points in outlines of boroughs 
outline_points = []
with io.open('resources/roads.geojson') as f:
    data = json.load(f) 
    coo = [i['geometry']['coordinates'] for i in data['features']]
    for outline in tqdm(coo):
        for series in outline:
            for line in series:
                for (a, b), (c, d) in zip(line, line[1:]):
                    outline_points.append([a * 0.758, b, c * 0.758, d])


HBox(children=(IntProgress(value=0, max=5), HTML(value='')))

In [11]:
pts = []

# Gets closest station location to point
def norm_loc(point):
    return min(name_to_location.values(), 
                  key=lambda x: (point[0] - x[0]) ** 2 + (point[1] - x[1]) ** 2)

# Appends chains of points in each subway line to pts
with io.open('resources/lines.csv', 'r') as f:
    for line in tqdm(list(f)[1:]):
        line = re.sub(r'.*LINESTRING \((.*)\).*', r'\1', line)
        pts.append([norm_loc([float(i.split(" ")[0]) * long_scale, float(i.split(" ")[1])]) for i in line.split(', ')])

# Dictionary mapping stations to connected stations
line_net = {k: set() for k in name_to_location.values()}
for line in tqdm(pts):
    for a, b in zip(line, line[1:]):
        line_net[a].add(b)
        line_net[b].add(a)

HBox(children=(IntProgress(value=0, max=742), HTML(value='')))

HBox(children=(IntProgress(value=0, max=742), HTML(value='')))

In [12]:
import networkx as nx
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d
import pandas as pd
from math import inf


def draw(matrix, color, filename, lines=False):
    '''Draw taxi traffic matrix with given a color and a filename to save
    to. If the lines parameter is set, draw the flow using shortest paths
    through the existing subway lines.
    '''
    fig = plt.figure(figsize=[24, 24])
    plt.rcParams['figure.figsize'] = [24, 24]

    # DDraw voronoi cells around each station
    vor = Voronoi([name_to_location[i] for i in name_to_location])
    voronoi_plot_2d(vor, show_vertices=False, line_colors='lightgray', show_points=False,
                     line_width=0.2, line_alpha=0.8, point_size=2)

    city = nx.Graph()
    
    # Draw subway stations with size from flow
    for i, (name, k, flow, location) in enumerate(station_tuples):
        city.add_node(name, pos=location, size=flow / 200000)
        
    def draw_outline():
        inc = 0
        # Draw boroughs
        for a, b, c, d in (outline_points):
            city.add_node(inc, pos=(a, b), size=0)
            inc -= 1
            city.add_node(inc, pos=(c, d), size=0)
            inc -= 1
            city.add_edge(inc + 1, inc + 2, weight='0.6', length=inf, color='orange')
        def dist(x, y):
            return sqrt((x[0] - y[0]) ** 2 + (x[1] - y[1]) ** 2)
        # Connect all of the subway lines
        for i in tqdm(line_net):
            for j in line_net[i]:
                n1 = [n for n, d in city.nodes(data=True) if d['pos'] == i]
                n2 = [n for n, d in city.nodes(data=True) if d['pos'] == j]
                for n1v, n2v in product(n1, n2):
                    city.add_edge(n1v, n2v, length=dist(i, j), weight=0.5, color=color)
    draw_outline()

    # For each pair of stations
    for i1, (name1, k1, j1, location1) in enumerate(station_tuples):
        for i2, (name2, k2, j2, location2) in enumerate(station_tuples):
            if lines:
                # If the lines parameter is set, increase the weight of the edges on
                # the shortest path between the two stations through valid subway lines
                # by an amount proportional to the respective value in the matrix.
                path = nx.shortest_path(city, source=name1, target=name2)
                for a, b in zip(path, path[1:]):
                    weight = matrix[i1][i2] ** 0.4
                    edge = city.get_edge_data(a, b, default=None)
                    if edge:
                        edge['weight'] += weight / len(matrix) / 2
            else:
                # Otherwise add a direct connection between the stations with weight 
                # proportional to the value in the point to point matrix.
                city.add_edge(name1, name2, weight=matrix[i1][i2] ** 0.9, color=color, length=inf)
            

    edgewidth = np.array([d['weight']
                          for (u, v, d) in city.edges(data=True)])
    edgecolor = np.array([d['color'] 
                          for (u, v, d) in city.edges(data=True)])

    pos = nx.get_node_attributes(city, 'pos')
    attr = nx.get_node_attributes(city, 'size')
    nodesize = [attr[i] for i in city.nodes()]
    
    nx.draw_networkx_nodes(city, pos, node_size=nodesize, node_color='red')
    nx.draw_networkx_edges(city, pos, width=edgewidth, edge_color=edgecolor)
    
    plt.rcParams['figure.facecolor'] = 'black'
    fig.patch.set_facecolor('black')
    plt.axis('off')
    plt.savefig(f'images/subway_model/{filename}_{year}.png', bbox_inches='tight', dpi=100, facecolor='black', pad_inches=0)

In [13]:
# Save computed p_matrix (rows normalized)
draw(solution, 'lightblue', 'subways')

# And with subway lines
draw(solution, 'lightblue', 'subways_lines', lines=True)

HBox(children=(IntProgress(value=0, max=308), HTML(value='')))

HBox(children=(IntProgress(value=0, max=308), HTML(value='')))

In [14]:
# Save non-normalized subway matrix
draw(solution_x / solution_x.sum() * len(solution_x), 'lightblue', 'subways_nn')

# And with subway lines
draw(solution_x / solution_x.sum() * len(solution_x), 'lightblue', 'subways_nn_lines', lines=True)

HBox(children=(IntProgress(value=0, max=308), HTML(value='')))

HBox(children=(IntProgress(value=0, max=308), HTML(value='')))

In [15]:
# Draw taxi traffic
draw(target_matrix, 'lightyellow', 'taxis')

HBox(children=(IntProgress(value=0, max=308), HTML(value='')))

In [16]:
# Draw absolute difference between "taxis" and "subways" plots
draw(abs(solution - target_matrix), 'violet', 'diff')

HBox(children=(IntProgress(value=0, max=308), HTML(value='')))